Image and Code Source : https://github.com/PacktPublishing/Hands-On-Natural-Language-Processing-with-PyTorch-1.x/tree/master/Chapter06

In [2]:
from torchtext import data
from torchtext import datasets
import torch
import spacy
import random
import numpy as np

import time

import torch.nn as nn
import torch.nn.functional as F

import spacy
nlp = spacy.load('en')

OSError: dlopen(/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so, 6): Symbol not found: __ZN3c104impl23ExcludeDispatchKeyGuardC1ENS_14DispatchKeySetE
  Referenced from: /anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so
  Expected in: /anaconda3/lib/python3.7/site-packages/torch/lib/libc10.dylib
 in /anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so

In [3]:
questions = data.Field(tokenize = 'spacy', batch_first = True)
labels = data.LabelField(dtype = torch.float)

In [5]:
train_data, _ = datasets.TREC.splits(questions, labels)

train_data, valid_data = train_data.split()

In [6]:
train_data

In [7]:
train_data.examples[0].text

['What', 'is', 'the', 'text', 'of', 'the', 'Fifth', 'Amendment', '?']

In [8]:
train_data.examples[0].label

'DESC'

In [9]:
print(len(train_data))
print(len(valid_data))

3816
1636


In [10]:
questions.build_vocab(train_data,
                 vectors = "glove.6B.200d", 
                 unk_init = torch.Tensor.normal_)

labels.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 398787/400000 [00:33<00:00, 11989.76it/s]

In [11]:
questions.vocab.vectors

tensor([[ 1.5142,  1.9301,  0.1088,  ...,  2.3204, -0.4258, -0.6275],
        [-0.4098, -1.6632, -0.7171,  ...,  0.0450,  0.2547,  0.1488],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.1940, -0.4843, -0.7601,  ...,  0.3863, -0.6567, -0.0112],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [-1.0700,  1.1445, -0.3129,  ...,  0.2329, -1.0910, -1.6553]])

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 64, 
    device = device)

cpu


In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        emb = self.embedding(text).unsqueeze(1)
        
        conved = [F.relu(c(emb)).squeeze(3) for c in self.convs]
                
        pooled = [F.max_pool1d(c, c.shape[2]).squeeze(2) for c in conved]
        
        concat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(concat)

In [ ]:
input_dimensions = len(questions.vocab)
output_dimensions = 6
embedding_dimensions = 200
pad_index = questions.vocab.stoi[questions.pad_token]

number_of_filters = 100
filter_sizes = [2,3,4]
dropout_pc = 0.5


model = CNN(input_dimensions, embedding_dimensions, number_of_filters, 
            filter_sizes, output_dimensions, dropout_pc, pad_index)

In [ ]:
glove_embeddings = questions.vocab.vectors

model.embedding.weight.data.copy_(glove_embeddings)

tensor([[ 1.9146, -1.0558,  0.2585,  ..., -1.7675, -1.1634,  0.1774],
        [-1.7950, -1.1161,  0.3654,  ...,  1.2729,  0.2299, -0.4907],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [-0.0364, -0.4738, -0.1913,  ...,  0.4871, -0.0841,  0.5200],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [ 1.9298, -0.7074, -0.0838,  ..., -0.9703, -1.3450, -1.7246]])

In [ ]:
unknown_index = questions.vocab.stoi[questions.unk_token]

model.embedding.weight.data[unknown_index] = torch.zeros(embedding_dimensions)
model.embedding.weight.data[pad_index] = torch.zeros(embedding_dimensions)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss().to(device)

model = model.to(device)

In [ ]:
def multi_accuracy(preds, y):
    pred = torch.max(preds,1).indices
    correct = (pred == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        preds = model(batch.text).squeeze(1)
        loss = criterion(preds, batch.label.long())
        
        acc = multi_accuracy(preds, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    total_epoch_loss = epoch_loss / len(iterator)
    total_epoch_accuracy = epoch_acc / len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            preds = model(batch.text).squeeze(1)
            
            loss = criterion(preds, batch.label.long())
            
            acc = multi_accuracy(preds, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    total_epoch_loss = epoch_loss / len(iterator)
    total_epoch_accuracy = epoch_acc / len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [ ]:
epochs = 10

lowest_validation_loss = float('inf')

for epoch in range(epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    if valid_loss < lowest_validation_loss:
        lowest_validation_loss = valid_loss
        torch.save(model.state_dict(), 'cnn_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {int(end_time - start_time)}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7s
	Train Loss: 1.197 | Train Acc: 52.70%
	 Val. Loss: 0.905 |  Val. Acc: 65.63%
Epoch: 02 | Epoch Time: 6s
	Train Loss: 0.757 | Train Acc: 73.71%
	 Val. Loss: 0.717 |  Val. Acc: 73.79%
Epoch: 03 | Epoch Time: 6s
	Train Loss: 0.523 | Train Acc: 82.48%
	 Val. Loss: 0.603 |  Val. Acc: 78.03%
Epoch: 04 | Epoch Time: 6s
	Train Loss: 0.351 | Train Acc: 89.77%
	 Val. Loss: 0.541 |  Val. Acc: 80.28%
Epoch: 05 | Epoch Time: 6s
	Train Loss: 0.235 | Train Acc: 93.67%
	 Val. Loss: 0.492 |  Val. Acc: 82.17%
Epoch: 06 | Epoch Time: 6s
	Train Loss: 0.164 | Train Acc: 95.60%
	 Val. Loss: 0.493 |  Val. Acc: 82.00%
Epoch: 07 | Epoch Time: 6s
	Train Loss: 0.120 | Train Acc: 97.14%
	 Val. Loss: 0.467 |  Val. Acc: 83.31%
Epoch: 08 | Epoch Time: 6s
	Train Loss: 0.082 | Train Acc: 98.37%
	 Val. Loss: 0.470 |  Val. Acc: 83.36%
Epoch: 09 | Epoch Time: 6s
	Train Loss: 0.054 | Train Acc: 99.05%
	 Val. Loss: 0.466 |  Val. Acc: 83.37%
Epoch: 10 | Epoch Time: 6s
	Train Loss: 0.048 | Train A

In [ ]:
model.load_state_dict(torch.load('cnn_model.pt'))

<All keys matched successfully>

In [ ]:
def predict_class(model, sentence, min_len = 5):
    
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [questions.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    
    model.eval()
    prediction = torch.max(model(tensor),1).indices.item()
    pred_index = labels.vocab.itos[prediction]
    
    return pred_index

In [ ]:
pred_class = predict_class(model, "How many roads must a man walk down?")
print('Predicted class is: ' + str(pred_class))

Predicted class is: NUM
